In [1]:
from scipy.linalg import expm
from quspin.operators import hamiltonian, commutator, exp_op # Hamiltonians and operators
from quspin.basis import tensor_basis, spin_basis_1d # bases
import numpy as np # general math functions
import matplotlib.pyplot as plt # plotting library

In [2]:
L = 4 #system size
M = 0.1 #fermion mass
N=1 #number of layers
l=1 #spin length
no_checks = dict(check_pcon=False,check_symm=False,check_herm=False)

In [3]:
#construct basis
basis=spin_basis_1d(L=L,S = str(l))
print(basis)

reference states: 
array index   /   Fock state   /   integer repr. 
      0.         |2 2 2 2>           80  
      1.         |2 2 2 1>           79  
      2.         |2 2 2 0>           78  
      3.         |2 2 1 2>           77  
      4.         |2 2 1 1>           76  
      5.         |2 2 1 0>           75  
      6.         |2 2 0 2>           74  
      7.         |2 2 0 1>           73  
      8.         |2 2 0 0>           72  
      9.         |2 1 2 2>           71  
     10.         |2 1 2 1>           70  
     11.         |2 1 2 0>           69  
     12.         |2 1 1 2>           68  
     13.         |2 1 1 1>           67  
     14.         |2 1 1 0>           66  
     15.         |2 1 0 2>           65  
     16.         |2 1 0 1>           64  
     17.         |2 1 0 0>           63  
     18.         |2 0 2 2>           62  
     19.         |2 0 2 1>           61  
     20.         |2 0 2 0>           60  
     21.         |2 0 1 2>           59  
     22

In [4]:
#initialisation of the Gauss laws

liste = []
liste_m = []
gauss_law_map = []
Gauss_law = []

for i in range(L+1):
    liste.append([[1.,i]])
    liste_m.append([[-1.,i]])
    if i == 0:
        gauss_law_map.append([
            ["z",liste[0]]
        ])
    
    elif (i == L) & (i%2 == 0):
        gauss_law_map.append([
            ["z",liste_m[i-1]]
        ])
    
    elif (i == L) & (i%2 == 1):
        gauss_law_map.append([
            ["z",liste_m[i-1]],
            ["I",liste[i-1]]
        ])        
        
    elif i%2 == 0:
        gauss_law_map.append([
            ["z",liste[i]],
            ["z",liste_m[i-1]],
        ])
    elif i%2 == 1:
        gauss_law_map.append([
            ["z",liste[i]],
            ["z",liste_m[i-1]],  
            ["I",liste[i]]
        ])
    Gauss_law.append(hamiltonian(gauss_law_map[i],dynamic_list=[],basis=basis,**no_checks))


In [5]:
#basis vectors as arrays
basis_vectors = []
for index in range(basis.Ns):
    state = np.zeros(basis.Ns)
    state[basis.Ns-index-1] = 1.
    state = state.tolist()
    basis_vectors.append(state)

In [6]:
#projectors on g=1 for every site
Proj = [np.zeros((basis.Ns,basis.Ns)) for i in range(L+1)]
for i in range(L+1):
    for state in basis_vectors:
        state = np.array(state)
        if (Gauss_law[i].expt_value(state)>=1.-1e-8) & (Gauss_law[i].expt_value(state)<=1.+1e-8):
            Proj[i] += np.outer(state,state)

In [7]:
#initialisation of the unitarily transformed Hamiltonian

linear_term = []
quadratic_term = []
interaction_p = []
interaction_m = []


for i in range(L-1):
    linear_term.append([[0.5/np.sqrt(2),i]])
    quadratic_term.append([[(-1)**i/np.sqrt(2),i,i+1]])
    interaction_p.append([
        ["+z", quadratic_term[i]],
        ["+", linear_term[i]]
    ])
    interaction_m.append([
        ["-z", quadratic_term[i]],
        ["-", linear_term[i]]
    ])

linear_term.append([[0.5/np.sqrt(2),L-1]])
interaction_p.append([
    ["+", linear_term[L-1]]
])
interaction_m.append([
    ["-", linear_term[L-1]]
])

kin_energy = [[0.5,i,i] for i in range(L)]
mass_term = [[2*(-1)**i*M,i] for i in range(L)]

kin_mass_map = [
            ["zz", kin_energy], 
            ["z", mass_term], 
]

H_int_p = []
H_int_m = []
interaction = [np.zeros((basis.Ns,basis.Ns)) for i in range(L)]
h_int = np.zeros((basis.Ns,basis.Ns))
for i in range(L):
    H_int_p.append(hamiltonian(interaction_p[i],dynamic_list=[],basis=basis,**no_checks))
    H_int_m.append(hamiltonian(interaction_m[i],dynamic_list=[],basis=basis,**no_checks))
    interaction[i] = (Proj[i]@H_int_p[i].toarray()@Proj[i+1]+Proj[i+1]@H_int_m[i].toarray()@Proj[i])

H_kin_mass = hamiltonian(kin_mass_map,dynamic_list=[],basis=basis,**no_checks)
h_kin_mass = H_kin_mass.toarray()

h_int = np.zeros((basis.Ns,basis.Ns),dtype = complex)

for i in range(L):
    h_int += interaction[i]


full_ham_matrix = h_kin_mass + h_int #Spin-1 Hamiltonian without the local contstaints


G=0
for i in range(L+1):
    G += 1e3*(Gauss_law[i]**2-Gauss_law[i])**2
constraint_full_matrix = G.toarray()

constrained_full_ham_matrix= full_ham_matrix + constraint_full_matrix #Spin-1 Hamiltonian with the local contstaints as a penalty term

In [8]:
#eigenvalues and eigenvectors of the Spin-1 Hamiltonian without the local contstaints
eigenval, eigenvec = np.linalg.eigh(full_ham_matrix)
print(eigenval[eigenval<=1e2])

[-0.84165027 -0.13898669 -0.13898669  0.03967222  0.03967222  0.210565
  0.33967222  0.33967222  0.41201398  0.6         0.6         0.6
  0.6         0.69336559  0.69336559  0.7         0.73967222  0.73967222
  0.7431159   0.9         0.9         0.9         0.9         1.
  1.          1.          1.          1.09336559  1.09336559  1.2
  1.26032778  1.26032778  1.3         1.3         1.3         1.3
  1.3         1.3         1.3         1.3         1.3         1.3
  1.37421746  1.38799188  1.4         1.43898669  1.43898669  1.56032778
  1.56032778  1.56589034  1.56589034  1.6         1.6         1.6
  1.6         1.7         1.7         1.7         1.7         1.7
  1.7         1.85502489  1.96032778  1.96032778  1.96589034  1.96589034
  2.          2.          2.          2.          2.1         2.1
  2.21521754  2.34350363  2.4         2.4         2.44074406  2.44074406
  2.8         2.84074406  2.84074406]


In [9]:
#eigenvalues and eigenvectors of the Spin-1 Hamiltonian with the local contstaints
eigenvalues, eigenvectors = np.linalg.eigh(constrained_full_ham_matrix)

In [10]:
#Gauge invariant part of the spectrum
print(eigenvalues[eigenvalues<=1e2])
print(eigenvectors[:,0])

[-0.84165027  0.210565    0.41201398  0.7         0.7431159   1.37421746
  1.38799188  1.85502489  2.21521754  2.34350363]
[-3.38577675e-21+0.j  2.85238554e-02+0.j  1.02629276e-18+0.j
 -1.42247325e-16+0.j  1.11022302e-16+0.j  1.94289029e-16+0.j
  5.55111512e-17+0.j -1.11022302e-16+0.j  0.00000000e+00+0.j
 -1.11022302e-16+0.j  1.44995330e-01+0.j  3.88578059e-16+0.j
 -2.19659360e-16+0.j -3.35001323e-01+0.j  1.49444062e-01+0.j
  0.00000000e+00+0.j -2.71050543e-19+0.j  0.00000000e+00+0.j
  5.09575021e-17+0.j  0.00000000e+00+0.j -5.42101086e-20+0.j
  0.00000000e+00+0.j  0.00000000e+00+0.j -1.65436123e-24+0.j
  5.04870979e-29+0.j  0.00000000e+00+0.j -1.15079808e-29+0.j
  0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
  0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
  0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
  4.78254770e-18+0.j -2.86532463e-01+0.j -2.38474992e-16+0.j
  3.78521696e-20+0.j  7.38470368e-01+0.j -3.35001323e-01+0.j
 -1.02075703e-17+0.j  1

In [11]:
#initial state
string = ""
for i in range(L):
    string +="0"
psi_0 = np.zeros(basis.Ns)
i_0 = basis.index(string)
psi_0[i_0] = 1.


In [12]:
#cost function of the initial state
expt_value = 0
matvec_h = full_ham_matrix@psi_0
matvec_c = constraint_full_matrix@psi_0
for i in range(basis.Ns):
    expt_value += np.conj(psi_0[i])*(matvec_h[i]+matvec_c[i])
print(np.real(expt_value))

4002.0


In [13]:
expt_value = 0
matvec = full_ham_matrix@psi_0
for i in range(basis.Ns):
    expt_value += np.conj(psi_0[i])*matvec[i]
print(expt_value)

(2+0j)


In [14]:
def clause(a,b,k,m,n):
    zero = 0
    m = int(m)
    n = int(n)
    for l in range(L):
        if l == k:
            if (int(a[l])!=m) | (int(b[l])!=n):
                zero +=1
        else:
            if a[l]!=b[l]:
                zero +=1
    return zero


In [15]:
#initialisation of the equatorial rotation sigma matrices 
sigma_01_temp = [np.zeros((basis.Ns,basis.Ns),dtype = complex) for i in range(L)]
sigma_12_temp = [np.zeros((basis.Ns,basis.Ns),dtype = complex) for i in range(L)]
sigma_02_temp = [np.zeros((basis.Ns,basis.Ns),dtype = complex) for i in range(L)]

sigma_z_01_temp = [np.zeros((basis.Ns,basis.Ns),dtype = complex) for i in range(L)]
sigma_z_12_temp = [np.zeros((basis.Ns,basis.Ns),dtype = complex) for i in range(L)]
sigma_z_02_temp = [np.zeros((basis.Ns,basis.Ns),dtype = complex) for i in range(L)]


for k in range(L):
    for i in range(basis.Ns):
        for j in range(basis.Ns):

            string_i = basis.int_to_state(basis.Ns-i-1,bracket_notation=False)
            string_j = basis.int_to_state(basis.Ns-j-1,bracket_notation=False)
            
            if (clause(string_i,string_j,k,0,1)==0):
                sigma_01_temp[k][i][j] = 0.5
            if (clause(string_i,string_j,k,1,0)==0):
                sigma_01_temp[k][i][j] = 0.5
                
            if (clause(string_i,string_j,k,1,2)==0):
                sigma_12_temp[k][i][j] = 0.5
            if (clause(string_i,string_j,k,2,1)==0):
                sigma_12_temp[k][i][j] = 0.5
                
            if (clause(string_i,string_j,k,0,2)==0):
                sigma_02_temp[k][i][j] = 0.5
            if (clause(string_i,string_j,k,2,0)==0):
                sigma_02_temp[k][i][j] = 0.5
                
            if (clause(string_i,string_j,k,0,0)==0):
                sigma_z_01_temp[k][i][j] = -1.
                sigma_z_02_temp[k][i][j] = -1.
                
            if (clause(string_i,string_j,k,1,1)==0):
                sigma_z_01_temp[k][i][j] = 1.
                sigma_z_12_temp[k][i][j] = -1.

            if (clause(string_i,string_j,k,2,2)==0):
                sigma_z_02_temp[k][i][j] = 1.
                sigma_z_12_temp[k][i][j] = 1.


sigma_01 = np.zeros((basis.Ns,basis.Ns),dtype = complex)
sigma_12 = np.zeros((basis.Ns,basis.Ns),dtype = complex)
sigma_02 = np.zeros((basis.Ns,basis.Ns),dtype = complex)

sigma_z_01 = np.zeros((basis.Ns,basis.Ns),dtype = complex)
sigma_z_12 = np.zeros((basis.Ns,basis.Ns),dtype = complex)
sigma_z_02 = np.zeros((basis.Ns,basis.Ns),dtype = complex)

for i in range(L):
    sigma_01 +=sigma_01_temp[i]
    sigma_12 +=sigma_12_temp[i]
    sigma_02 +=sigma_02_temp[i]
    sigma_z_01 +=sigma_z_01_temp[i]
    sigma_z_12 +=sigma_z_12_temp[i]
    sigma_z_02 +=sigma_z_02_temp[i]    


In [16]:
#initialisation of the Molmer-Sorensen gates
ms_01_temp = [np.zeros((basis.Ns,basis.Ns),dtype = complex) for i in range(L-1)]
ms_12_temp = [np.zeros((basis.Ns,basis.Ns),dtype = complex) for i in range(L-1)]
ms_02_temp = [np.zeros((basis.Ns,basis.Ns),dtype = complex) for i in range(L-1)]

for i in range(L-1):
    ms_01_temp[i] = ((sigma_01_temp[i]+sigma_01_temp[i+1])**2)/2  
    ms_12_temp[i] = ((sigma_12_temp[i]+sigma_12_temp[i+1])**2)/2  
    ms_02_temp[i] = ((sigma_02_temp[i]+sigma_02_temp[i+1])**2)/2  
    
ms_01 = np.zeros((basis.Ns,basis.Ns),dtype = complex)
ms_12 = np.zeros((basis.Ns,basis.Ns),dtype = complex)
ms_02 = np.zeros((basis.Ns,basis.Ns),dtype = complex)

for i in range(L-1):
    ms_01 +=ms_01_temp[i]
    ms_12 +=ms_12_temp[i]
    ms_02 +=ms_02_temp[i]

In [17]:
print(np.allclose(sigma_z_12_temp[0]@sigma_01_temp[0],sigma_01_temp[0]@sigma_z_12_temp[0],atol = 1e-8,rtol=1e-8))

False


In [18]:
def rot(sigma,theta):
    return expm(-1j*theta*sigma/2)

In [40]:
opt_params = []
function_values = []
def callback_function(x,fun,context):
    opt_params.append(x)
    function_values.append(fun)


In [45]:
#cost function definition
def cost_function_sigma(theta,psi,N):
    if len(theta)!=6*N+3:
        print("Warning: The number of variational parameters does not match the number of layers!")
    psi_var = psi
    for i in range(N):
        psi_var = rot(ms_02,theta[6*i+5])@rot(ms_12,theta[6*i+4])@rot(ms_01,theta[6*i+3])@rot(sigma_02,theta[6*i+2])@rot(sigma_12,theta[6*i+1])@rot(sigma_01,theta[6*i])@psi_var
    psi_var = rot(sigma_02,theta[6*N+2])@rot(sigma_12,theta[6*N+1])@rot(sigma_01,theta[6*N])@psi_var
    expt_value = 0
    matvec = full_ham_matrix@psi_var
    for i in range(basis.Ns):
        expt_value += np.conj(psi_var[i])*matvec[i]
    cost = expt_value
    return np.real(cost)

In [46]:
#optimisation algorithm
from scipy.optimize import minimize,dual_annealing
import datetime
duan_ranges = []
for i in range(6*N+3):
    duan_ranges.append((-10,10)) 
resduan = 0
print(datetime.datetime.now())
resduan = dual_annealing(cost_function_sigma, duan_ranges, args = (psi_0,N),maxiter = 1000,callback = callback_function )
print(datetime.datetime.now())


2022-02-27 20:31:28.371295
2022-02-27 20:37:12.105359


In [47]:
#results
print(resduan.fun)
print(resduan.x)
print(resduan.nit)

-0.5337821284361343
[-3.09428005  3.53001137 -4.3820812   0.0373266   4.14544425 10.
 -3.62545496 -2.17638647 10.        ]
1000


In [48]:
#fidelity of the result
theta = resduan.x
psi = psi_0

psi_var = psi_0
for i in range(N):
    psi_var = rot(ms_02,theta[6*i+5])@rot(ms_12,theta[6*i+4])@rot(ms_01,theta[6*i+3])@rot(sigma_02,theta[6*i+2])@rot(sigma_12,theta[6*i+1])@rot(sigma_01,theta[6*i])@psi_var
psi_var = rot(sigma_02,theta[6*N+2])@rot(sigma_12,theta[6*N+1])@rot(sigma_01,theta[6*N])@psi_var

print(np.abs(np.dot(np.conj(psi_var),eigenvectors[:,0]))**2)
#print(psi_var)
print(np.abs(np.dot(np.conj(psi_var),psi_var)))


0.7286763804334213
0.9999999999999988


In [177]:
Energy[3] = resduan.fun
Fidelity[3] = np.abs(np.dot(np.conj(psi_var),eigenvec[:,0]))**2

In [178]:
print(Energy,Fidelity)

[-0.66104042 -0.80008306 -0.80079807  0.          0.        ] [0.69923925 0.8206883  0.82184009 0.         0.        ]


In [131]:
Energy = np.zeros(5)
Fidelity = np.zeros(5)